In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        self.relu = nn.ReLU() # 课件代码虽然没显式定义self.relu但直接用了F.relu，两种方式皆可

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

# 实例化并测试
net = MLP()
X = torch.rand(2, 20) # 模拟输入数据
print(net(X))

tensor([[-0.1246, -0.1278, -0.0731, -0.1245, -0.2553, -0.0287, -0.2038,  0.0178,
         -0.1448,  0.2941],
        [-0.1326, -0.0939, -0.0174, -0.2706, -0.1974,  0.0147, -0.3340,  0.0060,
          0.0328,  0.0788]], grad_fn=<AddmmBackward0>)


In [3]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不参与求梯度的随机权重
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        # 控制流示例
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
print(net(X))

tensor(0.0573, grad_fn=<SumBackward0>)


In [4]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

# 组合成一个复杂的网络 Chimera
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
print(chimera(X))

tensor(-0.7443, grad_fn=<SumBackward0>)


In [5]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
output = net(X)
print(output)

# 访问第2层（索引从0开始）的参数字典
print(net[2].state_dict()) 

# 访问特定参数及其属性
print(type(net[2].bias))
print(net[2].bias.data)
print(net[2].bias) # 显示是否包含梯度信息

# 一次性访问所有参数
# 访问第0层的参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
# 访问整个网络的参数
print(*[(name, param.shape) for name, param in net.named_parameters()])

# 打印网络结构
print(net)

tensor([[0.3866],
        [0.4824]], grad_fn=<AddmmBackward0>)
OrderedDict([('weight', tensor([[ 0.3011, -0.2287, -0.2956,  0.0274,  0.0544, -0.2128,  0.0075, -0.2028]])), ('bias', tensor([0.3374]))])
<class 'torch.nn.parameter.Parameter'>
tensor([0.3374])
Parameter containing:
tensor([0.3374], requires_grad=True)
('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))
Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)


In [6]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 使用 add_module 添加子模块
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [7]:
# 定义初始化函数：正态分布初始化
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1)) # 重置网络
net.apply(init_normal)
print(net[0].weight.data[0], net[0].bias.data[0])

# 定义其他初始化函数
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

# 对不同的层应用不同的初始化
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.0019, -0.0007,  0.0081, -0.0086]) tensor(0.)
tensor([-0.4975,  0.0376,  0.0845,  0.1069])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [8]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, X):
        return X - X.mean()

layer = CenteredLayer()
print(layer(torch.FloatTensor([1, 2, 3, 4, 5])))

# 放入 Sequential 中使用
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())
Y = net(torch.rand(4, 8))
print(Y.mean())

tensor([-2., -1.,  0.,  1.,  2.])
tensor(-4.6566e-09, grad_fn=<MeanBackward0>)


In [9]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        # 包装成 Parameter，这样才会被模型识别为参数
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
        
    def forward(self, X):
        # 课件中使用了 .data 进行计算，注意：通常训练时直接用 self.weight 即可保留梯度
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

dense = MyLinear(5, 3)
print(dense.weight)
print(dense(torch.rand(2, 5)))

Parameter containing:
tensor([[-0.1142, -1.0486, -0.4494],
        [ 1.8148, -1.3738, -0.4287],
        [-1.9603, -0.8024,  1.4689],
        [ 0.2528, -0.3793,  1.5705],
        [ 0.5621, -0.7059,  1.6722]], requires_grad=True)
tensor([[0.0000, 0.0000, 0.6681],
        [0.2552, 0.0000, 0.7293]])


In [10]:
# 1. 保存和加载 Tensor
x = torch.arange(4)
torch.save(x, 'x-file')
x2 = torch.load('x-file')
print(x2)

# 2. 保存和加载字典
mydict = {'x': x, 'y': torch.zeros(4)} 
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
print(mydict2)

# 3. 保存和加载模型参数 (State Dict)
# 需要重新定义 MLP 类（如果上面没定义的话）
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

# 保存参数
torch.save(net.state_dict(), 'mlp.params')

# 加载参数到新模型
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval() # 切换到评估模式

print("Original output:", Y)
print("Clone output:", clone(X))

tensor([0, 1, 2, 3])
{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}
Original output: tensor([[ 0.6355,  0.1128, -0.2276, -0.6168,  0.3615, -0.0262,  0.0840, -0.5502,
          0.0218,  0.6671],
        [ 0.1656, -0.1170, -0.0556, -0.1937,  0.2017,  0.0535,  0.0728, -0.1172,
         -0.0865,  0.2482]], grad_fn=<AddmmBackward0>)
Clone output: tensor([[ 0.6355,  0.1128, -0.2276, -0.6168,  0.3615, -0.0262,  0.0840, -0.5502,
          0.0218,  0.6671],
        [ 0.1656, -0.1170, -0.0556, -0.1937,  0.2017,  0.0535,  0.0728, -0.1172,
         -0.0865,  0.2482]], grad_fn=<AddmmBackward0>)
